In [1]:
import requests, json, html5lib
import pandas as pd, pybaseball as pyb
from datetime import datetime as dt
from bs4 import BeautifulSoup as bs
import requests
from bs4 import BeautifulSoup

def clean_text(text):
    """Remove HTML tags and unnecessary whitespace."""
    if text:
        return ' '.join(text.split())
    return ""

def extract_link_text(html):
    """Extract text and URL from a given BeautifulSoup tag."""
    if html.find('a'):
        return {'text': clean_text(html.get_text()), 'url': html.find('a')['href']}
    return {'text': clean_text(html.get_text()), 'url': None}

# create function to parse game data df and set columns = Teams	Line	OU	W-L	Ov/Un/P	RF/RA	Starters	W-L	Team Rec	ERA	WHIP

def parse_game_data(rows):
    games = []
    for row in rows:
        game_details = {}
        columns = row.find_all('td')
        if len(columns) >= 18:  # Ensure there are enough columns for a complete dataset
            game_details['date'] = clean_text(columns[0].text)
            game_details['opponent'] = extract_link_text(columns[1])
            game_details['player'] = extract_link_text(columns[2])
            game_details['betting_line'] = clean_text(columns[3].text)
            game_details['over_under'] = clean_text(columns[4].text)
            game_details['result'] = clean_text(columns[5].text)
            game_details['win_loss'] = clean_text(columns[6].text)
            game_details['era'] = clean_text(columns[7].text)
            game_details['innings_pitched'] = clean_text(columns[8].text)
            game_details['hits_allowed'] = clean_text(columns[9].text)
            game_details['runs_allowed'] = clean_text(columns[10].text)
            game_details['earned_runs'] = clean_text(columns[11].text)
            game_details['walks'] = clean_text(columns[12].text)
            game_details['strikeouts'] = clean_text(columns[13].text)
            game_details['home_runs_allowed'] = clean_text(columns[14].text)
            games.append(game_details)
    return games

In [2]:

today = dt.today()
r = requests.get('https://www.statfox.com/mlb/gamematchup.asp')

In [3]:
soup = bs(r.text, 'html.parser')
tables= soup.find_all('table')


In [4]:
# Extract and parse the data from the relevant table
tables = soup.find_all('table', class_='datatable')
for table in tables:
    rows = table.find_all('tr')
    game_data = parse_game_data(rows[1:])  # Skip header row if present
    for game in game_data:
        df = pd.DataFrame(game, index=[0])

In [5]:
df

,date,opponent,player,betting_line,over_under,result,win_loss,era,innings_pitched,hits_allowed,runs_allowed,earned_runs,walks,strikeouts,home_runs_allowed
0,917: OAKLAND918: NY YANKEES,NaN,NaN,9-15 (L1)16-8 (W1),11-12-18-15-1,2.8 / 4.34.3 / 3.5,(L) A WOOD(L) N CORTES,0-21-1,2-3 (L1)2-3 (L2),7.893.41,2.0311.069,,"Thursday, 4/25/2024 (2:20 PM)Matchup Report | ...",919: HOUSTON920: CHICAGO CUBS,-120+110
